In [1]:
!pip install git+https://github.com/StochasticTree/stochtree.git

  Cloning https://github.com/StochasticTree/stochtree.git to /tmp/pip-req-build-l2547hoh
  Running command git clone --filter=blob:none --quiet https://github.com/StochasticTree/stochtree.git /tmp/pip-req-build-l2547hoh
  Resolved https://github.com/StochasticTree/stochtree.git to commit f55bbb47b57ef6160964084650ab81f557c9559c
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for stochtree: filename=stochtree-0.1.0-cp311-cp311-linux_x86_64.whl size=871293 sha256=0c3135825227cc3ee88d182f0403e6d57e8f7ab4a9bef6788ac62f7ad5406feb
  Stored in directory: /tmp/pip-ephem-wheel-cache-noqpdp9u/wheels/6b/16/bb/b09e1d07fb9c44bfd19200620859a0fdda75287afaa4a076bf
Successfully built stochtree


In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
def generate_did_data(
    n_units=200,
    num_pre_periods=5,
    num_post_periods=5,
    linearity_degree=1, # 1: fully linear, 2: half X non-linear, 3: treatment + all X non-linear
    propensity_coeffs={'intercept': 0.0, 'X1': 0.5, 'X7': -0.5}, # Coefficients for propensity score (using static X1 and X7)
    pre_trend_bias_delta=0.2,
    epsilon_scale=1,
    seed=42
):
    """
    Generates panel data for Difference-in-Differences analysis with controllable pre-trends,
    non-linearity, conditional treatment effects, and propensity score based treatment assignment.

    Generates exactly 8 covariates:
    - X1: Bernoulli(p=0.66) - Static (unit-level)
    - X2: Bernoulli(p=0.45) - Time-varying
    - X3: Normal(0,1) - Time-varying (Used in CATE)
    - X4: Normal(0,1) - Time-varying
    - X5: Normal(0,1) - Time-varying
    - X6: Normal(0,1) - Time-varying
    - X7: Normal(0,1) - Static (unit-level, Used in Propensity Score)
    - X8: Categorical{1,2,3,4} probs {0.3, 0.1, 0.2, 0.4} - Time-varying (Used in CATE)

    Args:
        n_units (int): Number of units (e.g., individuals, firms).
        num_pre_periods (int): Number of periods before treatment.
        num_post_periods (int): Number of periods after treatment.
        linearity_degree (int): Degree of linearity in the DGP:
            1: Fully linear Y, conditional treatment effect based on X3/X8.
            2: X1,X2,X3,X4 non-linear, conditional treatment effect based on X3/X8.
            3: All X covariates non-linear, conditional treatment effect based on X3/X8.
        propensity_coeffs (dict): Dictionary with coefficients for the propensity score calculation.
                                  Expected keys: 'intercept', 'X1', 'X7'.
                                  p = sigmoid(intercept + X1*coeff_X1 + X7*coeff_X7)
        pre_trend_bias_delta (float): Bias parameter to induce pre-trends in the treated group.
        epsilon_scale (float): Scale (standard deviation) of the error term.
        seed (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Generated panel data in long format with 'Y', covariates (X1-X8), 'treated_group',
                      'post_treatment', 'CATE', and 'propensity_score'.
    """
    np.random.seed(seed)
    total_periods = num_pre_periods + num_post_periods
    total_observations = n_units * total_periods
    unit_ids = np.arange(n_units)
    time_periods = np.arange(total_periods)

    # --- Fixed Covariate Betas ---
    beta_x = np.array([-0.75, 0.5, -0.5, -1.30, 1.8, 2.5, -1.0, 0.3])
    if len(beta_x) != 8:
        raise ValueError("beta_x must have exactly 8 elements.")

    # --- Set base treatment_effect_beta based on linearity_degree ---
    if linearity_degree == 1 or linearity_degree == 2:
        treatment_effect_beta = 0
    elif linearity_degree == 3:
        treatment_effect_beta = 0
    else:
        raise ValueError(f"linearity_degree ({linearity_degree}) must be 1, 2, or 3.")

    # --- Generate Static Covariates (Unit-Level) ---
    X1_unit = np.random.binomial(n=1, p=0.66, size=n_units) # Static Bernoulli for Propensity
    X7_unit = np.random.normal(0, 1, size=n_units)          # Static Numerical for Propensity

    # --- Propensity Score Calculation and Treatment Assignment ---
    z_unit = (propensity_coeffs['intercept'] +
              propensity_coeffs['X1'] * X1_unit +
              propensity_coeffs['X7'] * X7_unit)
    propensity_scores_unit = 1 / (1 + np.exp(-z_unit))
    treatment_assignment_random = np.random.uniform(0, 1, size=n_units)
    treated_units_mask = treatment_assignment_random < propensity_scores_unit
    treated_units = unit_ids[treated_units_mask]

    # --- Create Base DataFrame and Map Static Data ---
    data = pd.DataFrame({
        'unit_id': np.repeat(unit_ids, total_periods),
        'time': np.tile(time_periods, n_units)
    })
    unit_map = data['unit_id'].values # Index mapper from panel row to unit

    data['X1'] = X1_unit[unit_map]
    data['X7'] = X7_unit[unit_map]
    data['propensity_score'] = propensity_scores_unit[unit_map]
    data['treated_group'] = np.isin(data['unit_id'], treated_units).astype(int)

    # --- Generate Time-Varying Covariates (Panel-Level) ---
    data['X2'] = np.random.binomial(n=1, p=0.45, size=total_observations) # Time-varying Bernoulli
    data['X3'] = np.random.normal(0, 1, size=total_observations)         # Time-varying Numerical (for CATE)
    data['X4'] = np.random.normal(0, 1, size=total_observations)         # Time-varying Numerical
    data['X5'] = np.random.normal(0, 1, size=total_observations)         # Time-varying Numerical
    data['X6'] = np.random.normal(0, 1, size=total_observations)         # Time-varying Numerical

    # Time-varying Categorical (for CATE)
    cat_categories = [1, 2, 3, 4]
    cat_probabilities = [0.3, 0.1, 0.2, 0.4]
    data['X8'] = np.random.choice(cat_categories, size=total_observations, p=cat_probabilities)

    # --- Time indicators ---
    treatment_period = num_pre_periods
    data['post_treatment'] = np.where(data['time'] >= treatment_period, 1, 0)
    data['time_trend'] = data['time']

    # --- Generate error term ---
    data['epsilon'] = np.random.normal(scale=epsilon_scale, size=total_observations)

    # --- DGP parameters ---
    beta_0 = -0.5 # Intercept
    beta_treated = 0.75 # Main effect of treated group (alpha_i)
    beta_time = 0.2 # Main effect of time trend (gamma_t)

    # --- Calculate Conditional Treatment Effect (CATE) ---
    # Depends on X3 (first numerical, time-varying) and X8 (categorical, time-varying)
    sqrt_abs_X3 = np.sqrt(np.abs(data['X3']))
    cate_conditions = [
        (data['X8'] == 1) | (data['X8'] == 3),
        (data['X8'] == 2),
        (data['X8'] == 4)
    ]
    cate_choices = [
        treatment_effect_beta + 1.5 * sqrt_abs_X3,
        treatment_effect_beta,
        treatment_effect_beta - 0.5 * sqrt_abs_X3
    ]
    potential_cate = np.select(cate_conditions, cate_choices, default=treatment_effect_beta)



    actual_cate_contribution = potential_cate * data['treated_group'] * data['post_treatment']
    data['CATE'] = potential_cate # Store potential effect magnitude

    # --- Calculate Outcome Y based on Linearity Degree ---
    covariate_names = [f'X{i+1}' for i in range(8)]

    # Define non-linear functions for flexibility
    def nl_func1(x): return x**2
    def nl_func2(x): return np.exp(x / 2) # Scaled exp
    def nl_func3(x): return np.abs(x)
    def nl_func4(x): return np.sqrt(np.abs(x))

    cov_effect = 0

    if linearity_degree == 1: # Fully linear
        for i in range(8):
            cov_effect += beta_x[i] * data[covariate_names[i]]
        time_term = beta_time * data['time_trend']

    elif linearity_degree == 2: # X1, X2, X3, X4 non-linear
        # Apply specific non-linear functions to first 4 covariates
        cov_effect += beta_x[0] * nl_func1(data['X1']) # X1^2 (still 0 or 1)
        cov_effect += beta_x[1] * nl_func2(data['X2']) # exp(X2/2) (more distinct for 0/1)
        cov_effect += beta_x[2] * nl_func3(data['X3']) # abs(X3)
        cov_effect += beta_x[3] * nl_func4(data['X4']) # sqrt(abs(X4))
        # Linear term for remaining covariates
        for i in range(4, 8):
            cov_effect += beta_x[i] * data[covariate_names[i]]
        time_term = beta_time * data['time_trend']

    elif linearity_degree == 3: 
        # Apply different non-linear functions across all covariates
        nl_funcs = [nl_func1, nl_func2, nl_func3, nl_func4, nl_func1, nl_func2, nl_func3, nl_func4] # Example pattern
        for i in range(8):
            cov_effect += beta_x[i] * nl_funcs[i](data[covariate_names[i]])
        # Non-linear time trend
        time_term = beta_time * (data['time_trend']**2)

    # Combine all components for Y
    data['Y'] = (beta_0 + beta_treated * data['treated_group'] +
                 time_term + cov_effect +
                 actual_cate_contribution)

    # --- Add pre-trend bias ---
    if pre_trend_bias_delta != 0:
        pre_trend_effect = pre_trend_bias_delta * data['treated_group'] * (data['time'] - treatment_period) * (1 - data['post_treatment'])
        data['Y'] += pre_trend_effect

    # --- Add final error term ---
    data['Y'] += data['epsilon']

    # --- Final Touches ---
    # Reorder columns for clarity
    cols_order = ['unit_id', 'time', 'treated_group', 'post_treatment', 'propensity_score'] + \
                 covariate_names + ['CATE', 'Y']
    data = data[cols_order]

    return data


In [4]:
def find_first_treatment_indexes_array(df, min_time=4, eventually_treated=1):
    """
    Finds the indexes of the first row for each treatment group (0, 1, 2, 3)
    after filtering the DataFrame by time and eventually_treated, and returns them as a NumPy array.

    Args:
        df: The pandas DataFrame.
        min_time: The minimum time value.
        eventually_treated: The desired eventually_treated value.

    Returns:
        A NumPy array containing the first row indexes for each treatment group (0, 1, 2, 3),
        or None if no rows meet the criteria. Returns -1 if a treatment group does not appear in the filtered data.
    """

    filtered_df = df[(df['time'] >= min_time) & (df['eventually_treated'] == eventually_treated)]

    if filtered_df.empty:
        return None  # Return None if no rows match the time and eventually_treated criteria.

    indexes = []
    for group in [1, 2, 3]:
        group_df = filtered_df[filtered_df['treatment_group'] == group]
        if not group_df.empty:
            indexes.append(group_df.index[0])  # Get the first index
        else:
            indexes.append(-1) #Return -1 if the treatment group does not appear in the filtered data.

    return np.array(indexes)


In [5]:
from stochtree import BCFModel
from tqdm import tqdm  # Import tqdm for the progress bar

# Experiments

In [6]:
num_x_covariates = 6
linearity_degree=2

# Set the number of iterations and initialize the counter.
num_iterations = 100
count_at_least_two_non_significant = 0

num_pre_periods=4

num_post_periods=4
num_mcmc=500

epsilon_scale=1

RMSE_per_period=np.zeros([num_iterations,num_post_periods])
MAE_per_period=np.zeros([num_iterations,num_post_periods])
MAPE_per_period=np.zeros([num_iterations,num_post_periods])

list_accumulated_p_values=[]

RMSE_overall=np.zeros([num_iterations])
MAE_overall=np.zeros([num_iterations])
MAPE_overall=np.zeros([num_iterations])

for i in tqdm(range(num_iterations), desc="Progress", unit="iteration"):
    # Generate a random seed for each iteration.
    seed_val = i

    # Generate data with specified hyperparameters.
    data_linear = generate_did_data(
        n_units=200,
        linearity_degree=linearity_degree,
        num_pre_periods=num_pre_periods,
        num_post_periods=num_post_periods,
        pre_trend_bias_delta=0,
        epsilon_scale=epsilon_scale,
        seed=seed_val
    )
    data_linear['pi_hat']=0.5


    data_linear['D']=data_linear['post_treatment']*data_linear['treated_group']

    x_columns = [f"X{i}" for i in range(1, num_x_covariates + 1+2)]
    X = np.array(data_linear[["treated_group"] + x_columns +["time"]])

    Z=np.array(data_linear["D"])
    y=np.array(data_linear["Y"])


    bcf_model = BCFModel()
    general_params = {"keep_every": 5, "num_chains": 3}
    prognostic_forest_params = {"keep_vars": np.array([0, 1] + list(range(2, num_x_covariates + 3))+[num_x_covariates + 3])}
    treatment_effect_forest_params = {"keep_vars": np.array([3,8]+[num_x_covariates + 3])}
    bcf_model.sample(X_train=X, Z_train=Z, y_train=y, pi_train=np.array(data_linear['pi_hat']), num_gfr=50, num_mcmc=num_mcmc, general_params=general_params, prognostic_forest_params=prognostic_forest_params,
                treatment_effect_forest_params=treatment_effect_forest_params)

    CATE_indexes=data_linear[(data_linear['treated_group']==1) & (data_linear['post_treatment']==1)]['CATE'].index
    Pre_indexes=data_linear[(data_linear['treated_group']==1) & (data_linear['post_treatment']==0)]['CATE'].index
    true_CATE=data_linear[(data_linear['treated_group']==1) & (data_linear['post_treatment']==1)]['CATE']
    estimated_CATE=bcf_model.tau_hat_train.mean(axis=1)
    final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])

    for k in range(int(len(Pre_indexes)/4)):
        estimated_CATE_individual=estimated_CATE[CATE_indexes[4*k:4*(k+1)]]
        debias_term=0
        final_CATE[k,:]=estimated_CATE_individual-debias_term

    true_CATE=np.array(true_CATE).reshape(final_CATE.shape)

    num_mcmc=500
    final_CATE_pvalues=np.zeros([int(len(Pre_indexes)/4),num_post_periods,num_mcmc])

    accumulated_p_values=np.zeros([num_post_periods,int(len(Pre_indexes)/4)])

    for k in range(int(len(Pre_indexes)/4)):
      estimated_CATE_individual=bcf_model.tau_hat_train[CATE_indexes[4*k:4*(k+1)],:]
      debias_term=0
      final_CATE_pvalues[k,:,:]=estimated_CATE_individual-debias_term
      for h in range(num_post_periods):
        mean_values=final_CATE_pvalues[k,h,:]
        above_zero = np.sum(mean_values > 0)
        below_zero = np.sum(mean_values < 0)
        total_points = mean_values.size
        percentage_above_zero = (above_zero / total_points)
        percentage_below_zero = (below_zero / total_points)
        accumulated_p_values[h,k]=min(percentage_above_zero, percentage_below_zero)

    accumulated_p_values=accumulated_p_values.reshape(accumulated_p_values.shape[0]*accumulated_p_values.shape[1])
    list_accumulated_p_values.append(accumulated_p_values)


    RMSE_overall[i]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
    MAE_overall[i]=np.mean(np.abs(final_CATE-true_CATE))
    MAPE_overall[i]=np.mean(np.abs((final_CATE-true_CATE)/true_CATE))

    for h in range(num_post_periods):
        RMSE_per_period[i,h]=np.sqrt(np.mean((final_CATE[:,h]-true_CATE[:,h])**2))
        MAE_per_period[i,h]=np.mean(np.abs(final_CATE[:,h]-true_CATE[:,h]))
        MAPE_per_period[i,h]=np.mean(np.abs((final_CATE[:,h]-true_CATE[:,h])/true_CATE[:,h]))


mean_RMSE_overall=np.mean(RMSE_overall)
mean_MAE_overall=np.mean(MAE_overall)
mean_MAPE_overall=np.mean(MAPE_overall)
std_RMSE_overall=np.std(RMSE_overall)
std_MAE_overall=np.std(MAE_overall)
std_MAPE_overall=np.std(MAPE_overall)

print(f"Mean RMSE for {num_iterations} simulations: {mean_RMSE_overall}")
print(f"Standard Deviation RMSE for {num_iterations} simulations: {std_RMSE_overall}")
print(f"Mean MAE for {num_iterations} simulations: {mean_MAE_overall}")
print(f"Standard Deviation MAE for {num_iterations} simulations: {std_MAE_overall}")
print(f"Mean MAPE for {num_iterations} simulations: {mean_MAPE_overall}")
print(f"Standard Deviation MAPE for {num_iterations} simulations: {std_MAPE_overall}")

for h in range(num_post_periods):
    print(f"Mean RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(RMSE_per_period[:,h])}")
    print(f"Standard Deviation RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(RMSE_per_period[:,h])}")
    print(f"Mean MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAE_per_period[:,h])}")
    print(f"Standard Deviation MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAE_per_period[:,h])}")
    print(f"Mean MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAPE_per_period[:,h])}")
    print(f"Standard Deviation MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAPE_per_period[:,h])}")

Progress:   0%|          | 0/100 [00:00<?, ?iteration/s]<ipython-input-6-704685992a2c>:95: RuntimeWarning: divide by zero encountered in divide
  MAPE_overall[i]=np.mean(np.abs((final_CATE-true_CATE)/true_CATE))
<ipython-input-6-704685992a2c>:100: RuntimeWarning: divide by zero encountered in divide
  MAPE_per_period[i,h]=np.mean(np.abs((final_CATE[:,h]-true_CATE[:,h])/true_CATE[:,h]))
Progress:   1%|          | 1/100 [01:31<2:30:18, 91.10s/iteration]<ipython-input-6-704685992a2c>:95: RuntimeWarning: divide by zero encountered in divide
  MAPE_overall[i]=np.mean(np.abs((final_CATE-true_CATE)/true_CATE))
<ipython-input-6-704685992a2c>:100: RuntimeWarning: divide by zero encountered in divide
  MAPE_per_period[i,h]=np.mean(np.abs((final_CATE[:,h]-true_CATE[:,h])/true_CATE[:,h]))
Progress:   2%|▏         | 2/100 [02:56<2:23:42, 87.98s/iteration]<ipython-input-6-704685992a2c>:95: RuntimeWarning: divide by zero encountered in divide
  MAPE_overall[i]=np.mean(np.abs((final_CATE-true_CATE)/tr

Mean RMSE for 100 simulations: 0.36356489589057367
Standard Deviation RMSE for 100 simulations: 0.04362395604487175
Mean MAE for 100 simulations: 0.2857650601766634
Standard Deviation MAE for 100 simulations: 0.03860548877279258
Mean MAPE for 100 simulations: inf
Standard Deviation MAPE for 100 simulations: nan
Mean RMSE for 100 simulations for post-treatment period 1: 0.36438655459669334
Standard Deviation RMSE for 100 simulations for post-treatment period 1: 0.05366341567338631
Mean MAE for 100 simulations for post-treatment period 1: 0.2873483889708614
Standard Deviation MAE for 100 simulations for post-treatment period 1: 0.05029937938336003
Mean MAPE for 100 simulations for post-treatment period 1: inf
Standard Deviation MAPE for 100 simulations for post-treatment period 1: nan
Mean RMSE for 100 simulations for post-treatment period 2: 0.363755442209798
Standard Deviation RMSE for 100 simulations for post-treatment period 2: 0.0555540028179443
Mean MAE for 100 simulations for post


/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:185: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [7]:
# === Sheet 1: Metrics Data ===

# 1. Prepare data dictionary for metrics (overall and per_period)
metrics_data_dict = {
    'RMSE_overall': RMSE_overall,
    'MAE_overall': MAE_overall,
    'MAPE_overall': MAPE_overall,
}

# 2. Flatten the _per_period arrays into columns
for i in range(num_post_periods):
    metrics_data_dict[f'RMSE_period_{i}'] = RMSE_per_period[:, i]
    metrics_data_dict[f'MAE_period_{i}'] = MAE_per_period[:, i]
    metrics_data_dict[f'MAPE_period_{i}'] = MAPE_per_period[:, i]

# 3. Create the first DataFrame for metrics
df_metrics = pd.DataFrame(metrics_data_dict)

# === Sheet 2: P-Values Data (Handling Variable Lengths) ===

df_p_values = None # Initialize in case the list is empty

if not list_accumulated_p_values:
    print("Warning: 'list_accumulated_p_values' is empty. P-Value sheet will not be created.")
else:
    # 1. Find the maximum length of the p-value vectors
    max_len = 0
    for vec in list_accumulated_p_values:
        # Check if the element is actually a numpy array or list-like
        if hasattr(vec, '__len__'):
             max_len = max(max_len, len(vec))
        # else: handle potential non-iterable elements if necessary

    if max_len == 0 and list_accumulated_p_values:
         print("Warning: list_accumulated_p_values contains elements but none have length > 0.")
         # Decide how to handle this - maybe create an empty df?

    # 2. Create padded data
    padded_p_values = []
    for i, vec in enumerate(list_accumulated_p_values):
         # Ensure vec is treated as an iterable, default to empty if not applicable
        current_vec = []
        if hasattr(vec, '__len__'):
            current_vec = list(vec) # Convert numpy array to list for easy padding

        # Create a padded row with NaN for missing values
        padded_row = current_vec + [np.nan] * (max_len - len(current_vec))
        padded_p_values.append(padded_row)

    # 3. Create column names for the p-values sheet
    p_value_columns = [f'p_value_{i}' for i in range(max_len)]

    # 4. Create the second DataFrame for p-values
    df_p_values = pd.DataFrame(padded_p_values, columns=p_value_columns)

# === Save to Excel File with Multiple Sheets ===

output_filename_excel = 'adapted_BCF_CATE_PS_and_PValues_linearity=2.xlsx'

# Use ExcelWriter to write multiple DataFrames to different sheets
try:
    with pd.ExcelWriter(output_filename_excel, engine='openpyxl') as writer:
        # Write the metrics DataFrame to the first sheet
        df_metrics.to_excel(writer, sheet_name='Metrics', index=False, float_format='%.6f')
        print(f"Metrics data saved to sheet 'Metrics' in {output_filename_excel}")

        # Write the p-values DataFrame to the second sheet (if it exists)
        if df_p_values is not None:
            df_p_values.to_excel(writer, sheet_name='P_Values', index=False, float_format='%.6f')
            print(f"P-Values data saved to sheet 'P_Values' in {output_filename_excel}")
        else:
             # Optionally create an empty sheet or just skip
             print("P-Values sheet was not created as the source list was empty or contained no vectors.")


except ImportError:
    print("\nError: Cannot write Excel file. Please install the 'openpyxl' library.")
    print("You can install it using: pip install openpyxl")
except Exception as e:
    print(f"\nAn error occurred while writing the Excel file: {e}")



Metrics data saved to sheet 'Metrics' in adapted_BCF_CATE_PS_and_PValues_linearity=2.xlsx
P-Values data saved to sheet 'P_Values' in adapted_BCF_CATE_PS_and_PValues_linearity=2.xlsx
